# 1、create_sql_query_chain的使用

举例1：

In [ ]:
!pip install pymysql

In [2]:
# pip install -U langchain langchain-community langchain-openai
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

# 测试连接本地的mysql数据库
db_user = "root"
db_password = "123456"
db_host = "localhost" #或 127.0.0.1
db_port = "3306"
db_database = "db_diancan"
# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}")

print("操作的是哪种数据库：",db.dialect)
print("获取数据库中的表：",db.get_usable_table_names())

#执行查询操作
res = db.run("SELECT COUNT(*) FROM t_order")
print(res)

操作的是哪种数据库： mysql
获取数据库中的表： ['t_admin', 't_category', 't_dish', 't_order', 't_order_detail', 't_table', 't_unit']
[(40,)]


举例2：chain的使用

In [4]:
# pip install -U langchain langchain-community langchain-openai
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

#1、获取mysql数据库的连接
# 测试连接本地的mysql数据库
db_user = "root"
db_password = "123456"
db_host = "localhost" #或 127.0.0.1
db_port = "3306"
db_database = "db_diancan"
# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}")

# 2、获取大语言模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
chat_model = ChatOpenAI(model="gpt-4o-mini")

# 3、创建create_sql_query_chain的实例
chain = create_sql_query_chain(chat_model, db)
response = chain.invoke({"question": "数据表t_admin中一共有多少个管理员？",
                         "table_names_to_use":["t_admin"]})
print(response)

# 3、创建create_sql_query_chain的实例
chain = create_sql_query_chain(chat_model, db)
response = chain.invoke({"question": "一共有多少个管理员？",
                         })
print(response)


response = chain.invoke({"question": "数据表t_order中订单金额最高的是那个",
                         "table_names_to_use":["t_order"]})
print(response)

SQLQuery: SELECT COUNT(`id`) AS admin_count FROM t_admin;
```sql
SELECT COUNT(`id`) AS total_admins FROM t_admin;
```
SQLQuery: SELECT `order_no`, `sett_amount` FROM `t_order` ORDER BY `sett_amount` DESC LIMIT 1;


# 2、create_stuff_documents_chain的使用

In [5]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

# 定义提示词模板
prompt = PromptTemplate.from_template("""
基于文档{docs}中说的情况，香蕉是什么颜色的？
""")

# 创建链
llm = ChatOpenAI(model="gpt-4o-mini")
chain = create_stuff_documents_chain(llm, prompt, document_variable_name="docs")

# 文档输入
docs123 = [
    Document(
        page_content="苹果，学名Malus pumila Mill.，别称西洋苹果、柰，属于蔷薇科苹果属的植物。苹果是全球最广泛种植和销售的水果之一，具有悠久的栽培历史和广泛的分布范围。苹果的原始种群主要起源于中亚的天山山脉附近，尤其是现代哈萨克斯坦的阿拉木图地区，提供了所有现代苹果品种的基因库。苹果通过早期的贸易路线，如丝绸之路，从中亚向外扩散到全球各地。"
    ),
    Document(
        page_content="香蕉是白色的水果，主要产自热带地区。"

    ),
    Document(
        page_content="蓝莓是蓝色的浆果，含有抗氧化物质。"

    )
]
# 执行摘要
chain.invoke({"docs": docs123})

'根据你给出的信息，香蕉是白色的水果。不过，实际上，成熟的香蕉通常是黄色的。在一些情况下，它们也可能呈现绿色或褐色。因此，如果只根据你描述的内容，香蕉被称为白色的水果。'